In [85]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy.linalg as la
import math as mt


df=pd.read_csv("/ML101_train_dataset.csv")
df=df.dropna(subset=['Systolic BP','Diastolic BP'])

df = df[df['Diastolic BP'] < df['Systolic BP']]

df = df[(df['Systolic BP'] >= 0) & (df['Diastolic BP'] >= 0)]

Q1 = df['Systolic BP'].quantile(0.25)
Q3 = df['Systolic BP'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
median_value = df['Systolic BP'].median()
df['Systolic BP'] = df['Systolic BP'].apply(lambda x: median_value if x < lower_bound or x > upper_bound else x)

Q1 = df['Diastolic BP'].quantile(0.25)
Q3 = df['Diastolic BP'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
median_value = df['Diastolic BP'].median()
df['Diastolic BP'] = df['Diastolic BP'].apply(lambda x: median_value if x < lower_bound or x > upper_bound else x)

df.to_csv("cleaned_data.csv")
df_test=pd.read_csv("/ML101_dataset_test_feature.csv")




In [86]:
df_cleaned= pd.read_csv("cleaned_data.csv")

df_cleaned["Age"].fillna(df_cleaned["Age"].median(), inplace=True)

df_cleaned["Cholesterol level"].fillna(df_cleaned["Cholesterol level"].mean(), inplace=True)
df_cleaned["Blood Sugar level"].fillna(df_cleaned["Blood Sugar level"].mean(), inplace=True)
df_cleaned["Weight"].fillna(df_cleaned["Weight"].mean(), inplace=True)
df_cleaned["Height"].fillna(df_cleaned["Height"].mean(), inplace=True)
df_cleaned["Average Daily Steps"].fillna(df_cleaned["Average Daily Steps"].mean(), inplace=True)
df_cleaned["Caloric Intake"].fillna(df_cleaned["Caloric Intake"].mean(), inplace=True)
df_cleaned["Hours of Sleep"].fillna(df_cleaned["Hours of Sleep"].mean(), inplace= True)
df_cleaned=df_cleaned.dropna(subset=["LifeStyle"])
df_cleaned=df_cleaned.iloc[:,1:]
print(df_cleaned)


       Average Daily Steps  Hours of Sleep  Caloric Intake   Age  Gender  \
0                  15725.0        6.518042     1714.000000  56.0    Male   
1                   7424.0        4.856772     1366.000000  46.0    Male   
3                   7521.0        6.433183     2342.000000  48.0     NaN   
4                  16484.0        4.380561     2373.000000  25.0    Male   
5                  17563.0        4.195405     1547.000000  78.0  Female   
...                    ...             ...             ...   ...     ...   
61263               1774.0        6.315927     1856.000000  73.0    Male   
61265              14357.0        6.773970     2509.908829  53.0     NaN   
61266              16215.0        7.005686     1195.000000  44.0  Female   
61267              10959.0        9.661165     2233.000000  22.0    Male   
61268              18618.0        9.301179     3458.000000  37.0    Male   

           Height     Weight  Cholesterol level  Blood Sugar level  \
0      178.416872

In [87]:
import math as mt

def Most_Related(data_set, attribute_name):
    attribute_names=data_set.columns.values
    attribute_names=np.delete(attribute_names,4)
    attribute_names=attribute_names[0:10]
    target_arr=np.array(data_set[attribute_name])
    traget_mean=np.mean(target_arr)
    correlation_coff={}
    for i in attribute_names:
        attribute_series=np.array(data_set[i])
        attribute_mean=np.mean(attribute_series)
        coeff=abs(sum((target_arr-traget_mean)*(attribute_series-attribute_mean))/len(np.array(data_set)))
        coeff/=(np.std(attribute_series)*np.std(target_arr))
        correlation_coff[i]=coeff
    del correlation_coff["Systolic BP"]
    del correlation_coff["Diastolic BP"]
    print(correlation_coff)
    max_coff=max(correlation_coff.values())
    most_related=[]
    for i in correlation_coff:
        if correlation_coff[i]==max_coff:
            most_related.append(i)
            del correlation_coff[i]
            break
    max_coff=max(correlation_coff.values())
    for i in correlation_coff:
        if correlation_coff[i]==max_coff:
            most_related.append(i)
            del correlation_coff[i]
            break
    max_coff=max(correlation_coff.values())
    for i in correlation_coff:
        if correlation_coff[i]==max_coff:
            most_related.append(i)
            del correlation_coff[i]
            break
    return most_related


def Regression(data_set, input_variable_1, input_variable_2, input_variable_3, dependent_variable):
    X=np.vstack((data_set[input_variable_1],data_set[input_variable_2], data_set[input_variable_3],np.ones(len(data_set[input_variable_1])))).T
    y=np.array(data_set[dependent_variable])
    weight=la.inv(X.T@X)@X.T@y
    return weight


In [88]:
most_related=Most_Related(df_cleaned, "Systolic BP")


w1 =Regression(df_cleaned, most_related[0], most_related[1],most_related[2], "Systolic BP" )


predicted_systolic=[]
for i in zip(df_cleaned[most_related[0]],df_cleaned[most_related[1]],df_cleaned[most_related[2]]):
  predicted_systolic.append(w1[0]*i[0]+w1[1]*i[1]+w1[2]*i[2]+w1[3])
predicted_systolic=np.array(predicted_systolic)
actual_value=np.array(df_cleaned["Systolic BP"])

RMSE=mt.sqrt((sum((predicted_systolic-actual_value)**2))/len(predicted_systolic))
print("RMSE(Systolic):",RMSE)

most_related=Most_Related(df_cleaned, "Diastolic BP")

w2 =Regression(df_cleaned, most_related[0], most_related[1],most_related[2], "Diastolic BP" )

predicted_diastolic=[]
for i in zip(df_cleaned[most_related[0]],df_cleaned[most_related[1]],df_cleaned[most_related[2]]):
  predicted_diastolic.append(w2[0]*i[0]+w2[1]*i[1]+w2[2]*i[2]+w2[3])
predicted_diastolic=np.array(predicted_diastolic)
actual_value=np.array(df_cleaned["Diastolic BP"])

RMSE=mt.sqrt((sum((predicted_diastolic-actual_value)**2))/len(predicted_diastolic))
print("RMSE(Diastolic):", RMSE)

{'Average Daily Steps': 0.001632762871790961, 'Hours of Sleep': 0.001267396038390724, 'Caloric Intake': 0.6854008891109228, 'Age': 0.09373448577528029, 'Height': 0.028483964892483882, 'Weight': 0.07285970991572863, 'Cholesterol level': 0.3469191124724282, 'Blood Sugar level': 0.08811934277439917}
RMSE(Systolic): 19.478649360334966
{'Average Daily Steps': 0.0019706649254105373, 'Hours of Sleep': 0.0005244778826330256, 'Caloric Intake': 0.7023798028474267, 'Age': 0.14228949460941523, 'Height': 0.07035530865452011, 'Weight': 0.08238676076909922, 'Cholesterol level': 0.4563110987201556, 'Blood Sugar level': 0.15768218894077995}
RMSE(Diastolic): 8.921945778283622


In [89]:
predicted_systolic=[]

most_related=Most_Related(df_cleaned, "Systolic BP")

for i in zip(df_test[most_related[0]],df_test[most_related[1]],df_test[most_related[2]]):
  predicted_systolic.append(w1[0]*i[0]+w1[1]*i[1]+w1[2]*i[2]+w1[3])
predicted_systolic=np.array(predicted_systolic)

df_test["Systolic BP"]=predicted_systolic


predicted_diastolic=[]
most_related=Most_Related(df_cleaned, "Diastolic BP")
for i in zip(df_test[most_related[0]],df_test[most_related[1]],df_test[most_related[2]]):
  predicted_diastolic.append(w2[0]*i[0]+w2[1]*i[1]+w2[2]*i[2]+w2[3])
predicted_diastolic=np.array(predicted_diastolic)

df_test["Diastolic BP"]=predicted_diastolic


{'Average Daily Steps': 0.001632762871790961, 'Hours of Sleep': 0.001267396038390724, 'Caloric Intake': 0.6854008891109228, 'Age': 0.09373448577528029, 'Height': 0.028483964892483882, 'Weight': 0.07285970991572863, 'Cholesterol level': 0.3469191124724282, 'Blood Sugar level': 0.08811934277439917}
{'Average Daily Steps': 0.0019706649254105373, 'Hours of Sleep': 0.0005244778826330256, 'Caloric Intake': 0.7023798028474267, 'Age': 0.14228949460941523, 'Height': 0.07035530865452011, 'Weight': 0.08238676076909922, 'Cholesterol level': 0.4563110987201556, 'Blood Sugar level': 0.15768218894077995}


In [90]:
print(df_test)
df_test_new=df_test.iloc[:,9:]
print(df_test_new)
df_test_new["ID"]=np.arange(0,5000,1)
df_test_new.to_csv("Kaggle.csv",index=True)

      Average Daily Steps  Hours of Sleep  Caloric Intake  Age  Gender  \
0                   14459        4.787873            1422   56    Male   
1                    6646        6.144477            3381   69  Female   
2                   12495        9.864741            2864   46  Female   
3                   19544        6.454921            2952   32  Female   
4                    1650        4.849038            1687   60  Female   
...                   ...             ...             ...  ...     ...   
4995                16296        8.361073            3160   42  Female   
4996                12484        6.889406            1559   39  Female   
4997                18230        5.213463            2081   48  Female   
4998                11509        7.284685            3244   34    Male   
4999                15517        7.580033            2741   72  Female   

          Height     Weight  Cholesterol level  Blood Sugar level  \
0     170.117262  57.856498         222.76

In [91]:
from scipy.stats import multivariate_normal

df_cleaned=df_cleaned.drop(labels=["Gender"], axis=1)



def Covariance_Matrix(data):
    data=data.iloc[:,:10]
    means = np.mean(data, axis=0)
    num_obs = data.shape[0]
    covariance_matrix = (data - means).T.dot(data - means) / (num_obs - 1)

    return np.array(covariance_matrix)

def Mean(data):
    means = np.mean(data, axis=0)
    return means
def Class_Extraction(data_set, class_name):
    class_set=data_set.loc[data_set["LifeStyle"]==class_name]
    mean_arr=Mean(np.array(class_set.iloc[:,:10]))
    cov_mat=Covariance_Matrix(class_set)
    return mean_arr,cov_mat
classes=["Great", "Good", "Average", "Bad"]
mean_dict={}
cov_mat={}
for i in classes:
    mean_dict[i],cov_mat[i]=Class_Extraction(df_cleaned,i)

true_class_lables_train=np.array(df_cleaned.iloc[:,10])

class_probabilities_train={}
for i in true_class_lables_train:
    if i in class_probabilities_train:
        class_probabilities_train[i]+=1
    else:
        class_probabilities_train[i]=1
for i in class_probabilities_train:
    class_probabilities_train[i]/=len(np.array(df_cleaned))

predicted_class_lables=[]
df_test=df_test.drop(["Gender"], axis=1)
test_set=np.array(df_test)
train_set=np.array(df_cleaned.iloc[:,:10])
for i in train_set:
    predictions={}
    for j in classes:
        #likelihood=(1/(mt.pow((2*mt.pi),4.5)*mt.sqrt(determinant)))*mt.exp(-0.5(i-mean_dict[j])@np.inv(cov_mat[j])@(i-mean_dict[j]).T)
        likelihood=multivariate_normal(mean=mean_dict[j],cov=cov_mat[j]).pdf(i)
        p=likelihood*class_probabilities_train[j]
        predictions[j]=p
    max_prob=max(predictions.values())
    for j in predictions:
        if predictions[j]==max_prob:
            predicted_class_lables.append(j)
            break

predicted_class_lables=np.array(predicted_class_lables)



In [92]:
print(predicted_class_lables)
print(true_class_lables_train)
count=0
for i in range(len(predicted_class_lables)):
  if predicted_class_lables[i]==true_class_lables_train[i]:
    count+=1
print("Accuracy(training data)(%)=",count/len(predicted_class_lables)*100)

['Average' 'Bad' 'Bad' ... 'Good' 'Good' 'Bad']
['Average' 'Bad' 'Average' ... 'Great' 'Good' 'Bad']
Accuracy(training data)(%)= 76.73424280305089


In [93]:
predicted_class_lables=[]
for i in test_set:
    predictions={}
    for j in classes:
        #likelihood=(1/(mt.pow((2*mt.pi),4.5)*mt.sqrt(determinant)))*mt.exp(-0.5(i-mean_dict[j])@np.inv(cov_mat[j])@(i-mean_dict[j]).T)
        likelihood=multivariate_normal(mean=mean_dict[j],cov=cov_mat[j]).pdf(i)
        p=likelihood*class_probabilities_train[j]
        predictions[j]=p
    max_prob=max(predictions.values())
    for j in predictions:
        if predictions[j]==max_prob:
            predicted_class_lables.append(j)
            break

predicted_class_lables=np.array(predicted_class_lables)

df_test["LifeStyle"]=predicted_class_lables
print(df_test)
df_test_new_2=df_test["LifeStyle"]
df_test_new_2.to_csv("LifeStyle.csv")

      Average Daily Steps  Hours of Sleep  Caloric Intake  Age      Height  \
0                   14459        4.787873            1422   56  170.117262   
1                    6646        6.144477            3381   69  171.184727   
2                   12495        9.864741            2864   46  164.734031   
3                   19544        6.454921            2952   32  151.613392   
4                    1650        4.849038            1687   60  141.833224   
...                   ...             ...             ...  ...         ...   
4995                16296        8.361073            3160   42  168.474487   
4996                12484        6.889406            1559   39  181.845207   
4997                18230        5.213463            2081   48  158.024295   
4998                11509        7.284685            3244   34  172.973942   
4999                15517        7.580033            2741   72  151.342994   

         Weight  Cholesterol level  Blood Sugar level  Systolic

In [94]:
df_test_new_2["ID"]=np.arange(0,5000,1)
df_test_new_2.to_csv("LifeStyle.csv",index=True)

<ipython-input-94-1e66bd9b2540>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_new_2["ID"]=np.arange(0,5000,1)
